In [1]:
pip install --upgrade kaggle


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.4 MB/s  0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 17.2 MB/s  0:00:00m0:00:010:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  3/15 [pytokens]  WARNING: The script slugify is installed in '/usr/local/python/3.12.1/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━ 11/15 [mypy]]ec]  WARNING: The scripts dmypy, mypy, mypyc, stubgen and stubtest are installed in '/usr/local/python/3.12.1/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━ 13/15 [black]sdk]  WARNING: The scripts black and blackd are installed in '/usr/l

In [2]:
import pandas as pd
import numpy as np
import re
import string


In [3]:
def clean_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(r"http\S+", "", text)     # remove URLs
    text = re.sub(r"\d+", "", text)         # remove numbers
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = re.sub(r"\s+", " ", text).strip()
    return text


In [5]:
file_path = "../data/mental_disorders_reddit.csv"

chunks = pd.read_csv(
    file_path,
    usecols=["title", "selftext", "created_utc", "over_18", "subreddit"],
    chunksize=20000
)

processed_chunks = []


In [6]:
for chunk in chunks:
    # Rename columns
    chunk = chunk.rename(columns={
        "selftext": "post_text",
        "over_18": "nsfw"
    })

    # Combine title + body
    chunk["full_text"] = chunk["title"].fillna("") + " " + chunk["post_text"].fillna("")

    # Remove deleted / removed / empty
    chunk = chunk[
        chunk["full_text"].notna() &
        (chunk["full_text"].str.strip() != "") &
        (~chunk["full_text"].str.lower().str.contains("deleted")) &
        (~chunk["full_text"].str.lower().str.contains("removed"))
    ]

    # Clean text
    chunk["clean_text"] = chunk["full_text"].apply(clean_text)

    # Word count filter (THIS is what crashed before)
    chunk["word_count"] = chunk["clean_text"].apply(lambda x: len(x.split()))
    chunk = chunk[chunk["word_count"] >= 15]

    # Create topic label
    chunk["mental_health_topic"] = chunk["subreddit"].astype(str).str.lower()

    processed_chunks.append(
        chunk[[
            "clean_text",
            "mental_health_topic",
            "created_utc",
            "nsfw",
            "word_count"
        ]]
    )


In [7]:
df_clean = pd.concat(processed_chunks, ignore_index=True)
df_clean.shape


(570004, 5)

In [8]:
clean_path = "../data/mental_health_reddit_clean.csv"
df_clean.to_csv(clean_path, index=False)

print("✅ Clean NLP dataset saved to:", clean_path)


✅ Clean NLP dataset saved to: ../data/mental_health_reddit_clean.csv
